# Getting the data

In [1]:
import pandas as pd

In [2]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [3]:
df = df.iloc[:300]

In [4]:
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp


# Q1. Getting the embeddings model

In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
model_name = 'multi-qa-mpnet-base-dot-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
answer_llm = df.iloc[0].answer_llm

In [12]:
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [11]:
answer_llm_emb = model.encode(answer_llm)
answer_llm_emb[0]

/Users/Frank/Documentos/llm_zoomcamp/03_vector_search/vector_search_env/lib/python3.9/site-packages/transformers/models/mpnet/modeling_mpnet.py:1051: UserWarning: cumsum_out_mps supported by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/UnaryOps.mm:425.)
  incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask


-0.4224466

# Q2. Computing the dot product

In [14]:
df.iloc[0].answer_orig, df.iloc[0].answer_llm

('Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).')

In [15]:
import numpy as np
from tqdm import tqdm

answer_embeddings_list = []
evaluations = []

for index, row in tqdm(df.iterrows()):
    answer_llm = row.answer_llm
    answer_gt = row.answer_orig

    answer_llm_embeddings = model.encode(answer_llm)
    answer_gt_embeddings = model.encode(answer_gt)

    answer_embeddings_list.append((answer_llm_embeddings, answer_gt_embeddings))
    score = np.dot(answer_llm_embeddings, answer_gt_embeddings)
    evaluations.append(score)

300it [01:05,  4.60it/s]


In [16]:
np.percentile(evaluations, 75)

31.674306869506836

# Q3. Computing the cosine

In [28]:
len(answer_embeddings_list[0][1])

768

In [31]:
normalized_embeddings = []
for x in answer_embeddings_list:
    normalized_sublist = []
    for y in x:
        norm = np.linalg.norm(y)
        if norm == 0:
            normalized_sublist.append(y)
        else:
            normalized_sublist.append(y / norm)
    normalized_embeddings.append(normalized_sublist)

In [34]:
dot_products = []
for pair in normalized_embeddings:
    dot_product = np.dot(pair[0], pair[1])
    dot_products.append(dot_product)

In [35]:
np.percentile(dot_products, 75)

0.8362347036600113

# Q4. Rouge

In [36]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.iloc[10].to_dict()
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [37]:
r

{'answer_llm': "Yes, all sessions are recorded, so if you miss one, you won't miss anything. You can catch up on the content later. Additionally, you can submit your questions in advance for office hours, and those sessions are also recorded.",
 'answer_orig': 'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.',
 'document': '5170565b',
 'question': 'Are sessions recorded if I miss one?',
 'course': 'machine-learning-zoomcamp'}

In [40]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [39]:
scores['rouge-1']['f']

0.45454544954545456

# Q5. Average rouge score

In [44]:
np.mean([scores['rouge-1']['f'],scores['rouge-2']['f'],scores['rouge-l']['f']])

0.35490034990035496

# Q6. Average rouge score for all the data points

In [56]:
json_rouge = {
    'rouge_1': list(),
    'rouge_2': list(),
    'rouge_l': list()
}
for item in tqdm(df.itertuples()):
    scores = rouge_scorer.get_scores(item.answer_llm, item.answer_orig)[0]
    json_rouge['rouge_1'].append(scores['rouge-1']['f'])
    json_rouge['rouge_2'].append(scores['rouge-2']['f'])
    json_rouge['rouge_l'].append(scores['rouge-l']['f'])

300it [00:00, 444.43it/s]


In [58]:
df_rouge = pd.DataFrame(json_rouge)
df_rouge.head()

,rouge_1,rouge_2,rouge_l
0,0.095238,0.028169,0.095238
1,0.125000,0.055556,0.093750
2,0.415584,0.177778,0.389610
3,0.216216,0.047059,0.189189
4,0.142076,0.033898,0.120219


In [61]:
df_rouge.describe()['rouge_2']['mean']

0.20696501983423318